In [22]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import seaborn as sns # for nicer plots
import pandas as pd
from glob import glob
import re
from skimage.io import imread
import os
import keras
from keras.layers import Dense, Dropout, Activation, Flatten

#Pre-processing
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

#Measuring Accuracy
from sklearn.metrics import accuracy_score

%matplotlib inline
np.random.seed(123)         # for reproducibility

In [23]:
#Read the dataset

raw = pd.read_csv('train_labels.csv')
raw[980:]

,id,appliance
980,1980,5
981,1981,7
982,1982,1
983,1983,5
984,1984,1
985,1985,6
986,1986,9
987,1987,10


In [24]:
label1 = np.array(raw)
label = label1[:,1]
label[980:]


array([ 5,  7,  1,  5,  1,  6,  9, 10], dtype=int64)

In [25]:
import cv2
import numpy
import glob
import pylab as plt


#Input 1 (Train_voltage)

folders_voltage = glob.glob('data/train/train_voltage/*.png')

imagenames_list = []
train_voltage = []


for folder in folders_voltage:
    for f in glob.glob(folder):
        imagenames_list.append(f)

read_images = []        

for image1 in imagenames_list:
    image = cv2.imread(image1)
    image = cv2.resize(image, (128, 118))
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    #image = img_to_array(image)
    train_voltage.append(gray)
    

train_voltage = np.array(train_voltage)
train_voltage = train_voltage.reshape(train_voltage.shape[0], 128, 118, 1)
train_voltage.shape
imagenames_list[980:]


['data/train/train_voltage\\1980_v.png',
 'data/train/train_voltage\\1981_v.png',
 'data/train/train_voltage\\1982_v.png',
 'data/train/train_voltage\\1983_v.png',
 'data/train/train_voltage\\1984_v.png',
 'data/train/train_voltage\\1985_v.png',
 'data/train/train_voltage\\1986_v.png',
 'data/train/train_voltage\\1987_v.png']

In [26]:
#Input 2 (Train_current)

folders_current = glob.glob('data/train/train_current/*.png')

imagenames_list2 = []
train_current = []


for folder in folders_current:
    for f in glob.glob(folder):
        imagenames_list2.append(f)

read_images = []        

for image1 in imagenames_list2:
    image = cv2.imread(image1)
    image = cv2.resize(image, (128, 118))
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    #image = img_to_array(image)
    train_current.append(gray)
    

train_current = np.array(train_current)
train_current = train_current.reshape(train_current.shape[0], 128, 118, 1)
train_current.shape
imagenames_list2[980:]


['data/train/train_current\\1980_c.png',
 'data/train/train_current\\1981_c.png',
 'data/train/train_current\\1982_c.png',
 'data/train/train_current\\1983_c.png',
 'data/train/train_current\\1984_c.png',
 'data/train/train_current\\1985_c.png',
 'data/train/train_current\\1986_c.png',
 'data/train/train_current\\1987_c.png']

In [27]:
from sklearn.model_selection import train_test_split

#Convert dataset into train and test part (20% data as test set)
split = train_test_split(train_voltage, train_current, label,test_size=0.2,shuffle=False)

(Tr_voltage, Te_voltage, Tr_current, Te_current, Tr_label, Te_label) = split


In [28]:


print("train voltage: " , Tr_voltage.shape)
print("train current: " , Tr_current.shape)
print("first 10 labels: " , Tr_label[:10])


import tensorflow as tf 

# One-hot encode the labels
Tr_label = tf.keras.utils.to_categorical(Tr_label, 11)
print("train label: " , Tr_label.shape)



train voltage:  (790, 128, 118, 1)
train current:  (790, 128, 118, 1)
first 10 labels:  [ 8  1  8 10  6  1 10  2  0  5]
train label:  (790, 11)


In [29]:
print("test voltage: " , Te_voltage.shape)
print("test current: " , Te_current.shape)
print("second last 10 labels: " , Tr_label[:10])


import tensorflow as tf

# One-hot encode the labels
Te_label = tf.keras.utils.to_categorical(Te_label, 11)
print("test label: " , Te_label.shape)


test voltage:  (198, 128, 118, 1)
test current:  (198, 128, 118, 1)
second last 10 labels:  [[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
test label:  (198, 11)


In [30]:
import tensorflow as tf

# One-hot encode the labels
label = tf.keras.utils.to_categorical(label, 11)
print("test label: " , label.shape)

label[980:]

test label:  (988, 11)


array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [31]:
# Multiple Inputs
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

#Model
# first input model
voltage = Input(shape=(128,118,1))
conv11 = Conv2D(32, kernel_size=4, activation='relu')(voltage)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
conv13 = Conv2D(8, kernel_size=4, activation='relu')(pool12)
pool13 = MaxPooling2D(pool_size=(2, 2))(conv13)
flat1 = Flatten()(pool13)

# second input model
current = Input(shape=(128,118,1))
conv21 = Conv2D(32, kernel_size=4, activation='relu')(current)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
conv23 = Conv2D(8, kernel_size=4, activation='relu')(pool22)
pool23 = MaxPooling2D(pool_size=(2, 2))(conv23)
flat2 = Flatten()(pool23)


In [32]:
# merge input models
merge = concatenate([flat1, flat2])
# interpretation model
hidden1 = Dense(512, activation='relu')(merge)
hidden2 = Dense(256, activation='relu')(hidden1)
hidden3 = Dropout(0.5)(hidden2)
output = Dense(11, activation='softmax')(hidden3)
model = Model(inputs=[voltage, current], outputs=output)

# summarize layers
print(model.summary())

# plot graph
#plot_model(model, to_file='multiple_inputs.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 118, 1)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 128, 118, 1)  0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 125, 115, 32) 544         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 125, 115, 32) 544         input_4[0][0]                    
__________________________________________________________________________________________________
max_poolin

In [33]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


In [34]:
opt = keras.optimizers.nadam()
# Compile the model
model.compile(optimizer = 'Adam' , loss = "binary_crossentropy", metrics=["accuracy"])# Set a learning rate annealer

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [38]:
epochs = 100 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 64
early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 0, mode= 'min')
model_filepath='.weights.best.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(model_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [early_stopping, checkpoint]

In [39]:
from keras.preprocessing.image import ImageDataGenerator


# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 40)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=666)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=666)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
gen_flow = gen_flow_for_two_inputs(Tr_voltage, Tr_current, Tr_label)

In [40]:
# Fit the model using our generator defined above
model.fit_generator(gen_flow, validation_data=([Te_voltage, Te_current],Te_label),steps_per_epoch= len(Tr_voltage) / batch_size,
                    epochs=100,callbacks=callbacks_list)

Epoch 1/100
13/12 [===============================] - 2s 146ms/step - loss: 0.0377 - acc: 0.9875 - val_loss: 0.1030 - val_acc: 0.9679

Epoch 00001: val_loss improved from inf to 0.10296, saving model to .weights.best.hdf5
Epoch 2/100
13/12 [===============================] - 2s 127ms/step - loss: 0.0308 - acc: 0.9894 - val_loss: 0.0879 - val_acc: 0.9720

Epoch 00002: val_loss improved from 0.10296 to 0.08791, saving model to .weights.best.hdf5
Epoch 3/100
13/12 [===============================] - 2s 123ms/step - loss: 0.0229 - acc: 0.9918 - val_loss: 0.0999 - val_acc: 0.9660

Epoch 00003: val_loss did not improve from 0.08791
Epoch 4/100
13/12 [===============================] - 2s 127ms/step - loss: 0.0184 - acc: 0.9942 - val_loss: 0.1032 - val_acc: 0.9706

Epoch 00004: val_loss did not improve from 0.08791
Epoch 5/100
13/12 [===============================] - 2s 129ms/step - loss: 0.0211 - acc: 0.9925 - val_loss: 0.1100 - val_acc: 0.9706

Epoch 00005: val_loss did not improve from 0.

In [41]:
#H = model.fit([train_voltage,train_current], label,
#              epochs=epochs,verbose=1,validation_split = 0.2, callbacks=callbacks_list)

# Test

In [42]:
#Input 1 (Test_voltage)

folders_voltage_test = glob.glob('data/test/test_voltage/*.png')

imagenames_list_test = []
test_voltage = []


for folder in folders_voltage_test:
    for f in glob.glob(folder):
        imagenames_list_test.append(f)

read_images = []        

for image1 in imagenames_list_test:
    image = cv2.imread(image1)
    image = cv2.resize(image, (128, 118))
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    #image = img_to_array(image)
    test_voltage.append(gray)
    

test_voltage = np.array(test_voltage)
test_voltage = test_voltage.reshape(test_voltage.shape[0], 128, 118, 1)
print(test_voltage.shape)
imagenames_list_test[:10]


(659, 128, 118, 1)


['data/test/test_voltage\\1988_v.png',
 'data/test/test_voltage\\1989_v.png',
 'data/test/test_voltage\\1990_v.png',
 'data/test/test_voltage\\1991_v.png',
 'data/test/test_voltage\\1992_v.png',
 'data/test/test_voltage\\1993_v.png',
 'data/test/test_voltage\\1994_v.png',
 'data/test/test_voltage\\1995_v.png',
 'data/test/test_voltage\\1996_v.png',
 'data/test/test_voltage\\1997_v.png']

In [43]:
#Input 2 (Test_current)

folders_voltage_testc = glob.glob('data/test/test_current/*.png')

imagenames_list_testc = []
test_current = []


for folder in folders_voltage_testc:
    for f in glob.glob(folder):
        imagenames_list_testc.append(f)

read_images = []        

for image1 in imagenames_list_testc:
    image = cv2.imread(image1)
    image = cv2.resize(image, (128, 118))
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    #image = img_to_array(image)
    test_current.append(gray)
    

test_current = np.array(test_current)
test_current = test_current.reshape(test_current.shape[0], 128, 118, 1)
print(test_current.shape)
imagenames_list_testc[:10]


(659, 128, 118, 1)


['data/test/test_current\\1988_c.png',
 'data/test/test_current\\1989_c.png',
 'data/test/test_current\\1990_c.png',
 'data/test/test_current\\1991_c.png',
 'data/test/test_current\\1992_c.png',
 'data/test/test_current\\1993_c.png',
 'data/test/test_current\\1994_c.png',
 'data/test/test_current\\1995_c.png',
 'data/test/test_current\\1996_c.png',
 'data/test/test_current\\1997_c.png']

In [44]:
# predict results
results = model.predict([test_voltage,test_current])

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)
results[:10]

array([8, 1, 2, 8, 2, 5, 8, 3, 2, 0], dtype=int64)

In [45]:
results = pd.Series(results,name="appliance")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission_format.csv",index=False)